# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Singleton Pattern

**Subtopic** - Singleton used for configuration: a Config class reads file once globally accessible one instance

**Use Case** - Implement a Config singleton class in Python that reads a JSON configuration file once and provides globally accessible configuration parameters. The class should be able to handle the absence or unavailability of the configuration file and provide default configuration parameters in such cases.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
Pyro4>=4.80,<5.0
```


# Prompt

# Problem Description

You need a thread-safe Singleton `Config` class that builds its settings from four layers provided entirely via JSON input (no file or real environment access):

1. **Defaults**:  a built-in dict.
2. **Config JSON** : the contents of what would have been `config.json`.
3. **Environment overrides** : key/value pairs simulating `APP_` env vars.
4. **Command-line overrides** : highest-priority dict.

The class must support nested keys via dot-notation (e.g. `"database.host"`) and provide read-only methods (`get`, `get_int`, `get_bool`). Reads must be lock-free; all merges and a `reload()` must be guarded by an `RLock`.

# Input Format

All inputs come as one JSON object, for example:

```json
{
  "defaults": {
    "server": { "host": "127.0.0.1", "port": 8000 },
    "logging": { "level": "INFO" }
  },
  "config_json": {
    "server": { "port": 9000 },
    "feature_flags": { "beta": true }
  },
  "env_vars": {
    "server__host": "0.0.0.0",
    "feature_flags__beta": "false"
  },
  "cli_overrides": {
    "server__port": 8081,
    "new_feature": true
  }
}
```

# Output Format

The `Config` singleton exposes:

```python
cfg.get("server.host")            # "0.0.0.0"
cfg.get_int("server.port")        # 8081
cfg.get_bool("feature_flags.beta")# False
cfg.get("new_feature")            # True
```

# Examples

Given the above input, the final merged config is:

```json
{
  "server": { "host": "0.0.0.0", "port": 8081 },
  "logging": { "level": "INFO" },
  "feature_flags": { "beta": false },
  "new_feature": true
}
```


# Requirements


# Explicit Requirements

1. Singleton accessed via:

   ```python
   cfg = Config.get_instance(input_json)
   ```
2. Merge order: defaults < config_json < env_vars < cli_overrides.
3. Use `threading.RLock` for all writes and `reload()`.
4. Reads (`get`, `get_int`, `get_bool`) must never acquire the lock.
5. Support nested lookups with dot-notation.
6. Type helpers must safely cast or return the provided default.
7. `reload()` re-applies layers 2–3 (preserving cli overrides) under lock.

# Implicit Requirements

* Values may be str, int, float, bool, list, or dict.
* Env var names use `__` to denote nesting and always override JSON, even if empty string.
* CLI overrides are permanent and survive `reload()`.
* Unknown keys in JSON or env must be added dynamically.
* Keys are case-sensitive.

# Function Signature

```python
class Config:
    @classmethod
    def get_instance(cls, input_json: dict) -> "Config": ...
    def get(self, key: str, default=None) -> Any: ...
    def get_int(self, key: str, default: int | None = None) -> int | None: ...
    def get_bool(self, key: str, default: bool | None = None) -> bool | None: ...
    def reload(self) -> None: ...
```

# Constraints

* Total merge time ≤ 50 ms.
* Memory footprint ≤ 5 MB.
* Concurrent `reload()` calls never raise or return partial data.
* Reader threads block < 1 ms during reload.
* All override keys must be valid identifiers or use `__`.

# Edge Cases

* `config_json` is `null` or empty.
* `config_json` contains malformed types.
* Env var refers to missing nested path (auto-create).
* CLI override supplies scalar where dict expected.
* Simultaneous `get_instance()` calls from many threads.

# Solution Expectation

Implement lazy instantiation in `get_instance()` guarded by an `RLock`. Internally keep:

* `_base` = deep copy of `defaults`
* `_json_layer` = deep merge of `_base` + `config_json`
* `_env_layer` = deep merge of `_json_layer` + parsed `env_vars`
* `_overrides` = `cli_overrides` (always reapplied last)

On startup: Build `_config = merge(_env_layer, _overrides)`.
- `get()` splits `key` on `"."` and walks `_config`.
- `reload()` acquires lock, rebuilds `_json_layer` and `_env_layer` from input
JSON, merges `_config = merge(_env_layer, _overrides)`, then releases lock.
Provide a private `_deep_merge(dest, src)` that recursively merges dicts.



In [ ]:
# code
"""
Minimal SOA sample used by the test-suite.

• ArithmeticService & StatsService – API-key auth
• ConfigHolder – in-memory, thread-safe
• ServiceRegistry – registers services + 30 s heartbeat
• ServiceProxy – retry, back-off, NS fallback, async metrics
"""

from __future__ import annotations

import logging
import threading
import time
import traceback
from concurrent.futures import ThreadPoolExecutor
from typing import Any, Dict, List, Optional

import Pyro4

# ────────── Exceptions ──────────────────────────────────────────
class AuthenticationError(Exception):
    """Raised when an invalid API key is supplied."""


class ServiceError(Exception):
    """Wraps remote/network exceptions with extra context."""

    def __init__(self, msg: str, svc: str, ver: str, mth: str, code: str, tb: str = ""):
        super().__init__(msg)
        self.service, self.version, self.method, self.code = svc, ver, mth, code
        self.traceback = tb


# ────────── Services ────────────────────────────────────────────
@Pyro4.expose
class ArithmeticService:
    """Simple arithmetic RPCs secured by API key."""

    def __init__(self, api_key: str, logger: Optional[logging.Logger] = None):
        self._key = api_key
        self._log = logger or logging.getLogger("arith")

    def _check(self, key: str):
        if key != self._key:
            raise AuthenticationError("invalid key")

    def add(self, a, b, api_key):       self._check(api_key); return a + b
    def subtract(self, a, b, api_key):  self._check(api_key); return a - b
    def multiply(self, a, b, api_key):  self._check(api_key); return a * b

    def divide(self, a, b, api_key):
        self._check(api_key)
        if b == 0:
            raise ZeroDivisionError("division by zero")
        return a / b


@Pyro4.expose
class StatsService:
    """Collects call metrics."""

    def __init__(self, api_key: str, logger: Optional[logging.Logger] = None):
        self._key = api_key
        self._log = logger or logging.getLogger("stats")
        self.calls: List[tuple] = []

    def _check(self, key: str):
        if key != self._key:
            raise AuthenticationError("invalid key")

    def record_call(self, svc, ver, mth, dur, api_key):
        self._check(api_key)
        self.calls.append((svc, ver, mth, dur))


# ────────── Config holder (pure in-memory) ───────────────────────
class ConfigHolder:
    """Thread-safe dict with minimal defaults (no file I/O)."""

    def __init__(self, data: Optional[Dict[str, Any]] = None):
        self._lock = threading.RLock()
        self._data = data or {
            "auth": {"api_key": "dummy"},
            "name_servers": [],
            "services": {},
            "client": {"retry_count": 0, "timeout_seconds": 1, "backoff_factor": 0},
        }

    def get(self) -> Dict[str, Any]:
        with self._lock:
            return self._data.copy()

    def set(self, new_data: Dict[str, Any]):
        with self._lock:
            self._data = new_data.copy()


# ────────── Service registry & heartbeat ─────────────────────────
class ServiceRegistry:
    """Registers services and re-registers every 30 s."""

    def __init__(self, cfg: ConfigHolder):
        self.cfg = cfg
        self.d = Pyro4.Daemon(host="127.0.0.1")
        self.reg: List[tuple[str, Any]] = []
        self._register_all()
        threading.Thread(target=self._heartbeat, daemon=True).start()

    # internal helpers ------------------------------------------------
    def _ns(self):
        for addr in self.cfg.get().get("name_servers", []):
            host, port = addr.split(":")
            try:
                return Pyro4.locateNS(host=host, port=int(port))
            except Pyro4.errors.PyroError:
                pass
        return None

    def _register(self, name: str, obj: Any):
        uri = self.d.register(obj)
        ns = self._ns()
        if ns:
            ns.register(name, uri)
        self.reg.append((name, obj))

    def _register_all(self):
        key = self.cfg.get()["auth"]["api_key"]
        self._register("arithmetic.service.v1_0", ArithmeticService(key))
        self._register("stats.service.v1_0", StatsService(key))

    def _heartbeat(self):
        while True:
            time.sleep(30)
            ns = self._ns()
            if not ns:
                continue
            for name, obj in self.reg:
                ns.register(name, self.d.register(obj))


# ────────── Client proxy ─────────────────────────────────────────
class ServiceProxy:
    """Retry, back-off, NS fallback, async metrics."""

    def __init__(self, cfg: Dict[str, Any]):
        self.cfg, self.key = cfg, cfg["auth"]["api_key"]
        cl = cfg["client"]
        self.retries, self.back, self.timeout = cl["retry_count"], cl["backoff_factor"], cl["timeout_seconds"]
        self.pool = ThreadPoolExecutor(2)

    # helpers --------------------------------------------------------
    def _ns(self):
        for addr in self.cfg["name_servers"]:
            h, p = addr.split(":")
            try:
                return Pyro4.locateNS(host=h, port=int(p))
            except Pyro4.errors.PyroError:
                pass
        return None

    def _px(self, svc: str, ver: str):
        name = f"{svc}.service.v{ver}"
        ns = self._ns()
        uri = ns.lookup(name) if ns else self.cfg["services"][svc]["direct_uris"][ver][0]
        p = Pyro4.Proxy(uri)
        p._pyroTimeout = self.timeout
        return p

    # public call ----------------------------------------------------
    def call(self, svc, ver, mth, args, kw=None):
        kw = dict(kw or {}, api_key=self.key)
        for i in range(self.retries + 1):
            try:
                p = self._px(svc, ver)
                t0 = time.time()
                res = getattr(p, mth)(*args, **kw)
                dur = (time.time() - t0) * 1000
                self.pool.submit(self._metric, svc, ver, mth, dur)
                return {"result": res, "duration_ms": dur}
            except (ZeroDivisionError, Pyro4.errors.PyroError) as exc:
                if i == self.retries:
                    raise ServiceError(str(exc), svc, ver, mth, exc.__class__.__name__, traceback.format_exc())
                time.sleep(self.back * (2 ** i))

    def _metric(self, svc, ver, mth, dur):
        try:
            self._px("stats", "1_0").record_call(svc, ver, mth, dur, self.key)
        except Exception:
            pass


In [ ]:
# tests
import unittest
import threading
import time
from unittest.mock import patch

import Pyro4
import main


# ───── helper to spin up a fresh daemon per test ────────────
def make_fixture():
    key = "secret"
    daemon = Pyro4.Daemon(host="127.0.0.1")
    arith = main.ArithmeticService(key)
    stats = main.StatsService(key)
    uri_a = daemon.register(arith)
    uri_s = daemon.register(stats)
    th = threading.Thread(target=daemon.requestLoop, daemon=True)
    th.start()

    cfg = {
        "name_servers": ["localhost:9999"],
        "services": {
            "arithmetic": {"direct_uris": {"v1_0": [str(uri_a)]}},
            "stats": {"direct_uris": {"1_0": [str(uri_s)]}},
        },
        "auth": {"api_key": key},
        "client": {"retry_count": 1, "timeout_seconds": 0.5, "backoff_factor": 0.1},
    }
    proxy = main.ServiceProxy(cfg)
    return daemon, th, cfg, proxy, stats


class TestSOA(unittest.TestCase):
    # ───────── local logic ──────────────────────────
    def test_local_ops(self):
        a = main.ArithmeticService("k")
        self.assertEqual(a.add(2, 3, "k"), 5)
        with self.assertRaises(main.AuthenticationError):
            a.add(1, 1, "bad")

    def test_local_zero(self):
        a = main.ArithmeticService("k")
        with self.assertRaises(ZeroDivisionError):
            a.divide(1, 0, "k")

    # ───────── remote happy path ─────────────────────
    def test_remote_add(self):
        d, t, _, px, _ = make_fixture()
        self.assertEqual(px.call("arithmetic", "v1_0", "add", [4, 6])["result"], 10)
        d.shutdown(); t.join(timeout=0.1)

    # ───────── remote error paths ────────────────────
    def test_remote_div_zero(self):
        d, t, _, px, _ = make_fixture()
        with self.assertRaises(main.ServiceError):
            px.call("arithmetic", "v1_0", "divide", [1, 0])
        d.shutdown(); t.join(timeout=0.1)

    def test_remote_bad_key(self):
        d, t, cfg, _, _ = make_fixture()
        wrong = {**cfg, "auth": {"api_key": "bad"}}
        with self.assertRaises(main.ServiceError):
            main.ServiceProxy(wrong).call("arithmetic", "v1_0", "add", [1, 1])
        d.shutdown(); t.join(timeout=0.1)

    # ───────── metrics posting ───────────────────────
    def test_metrics(self):
        d, t, _, px, stats = make_fixture()
        stats.calls.clear()
        px.call("arithmetic", "v1_0", "subtract", [7, 2])
        time.sleep(0.2)
        self.assertTrue(any(c[2] == "subtract" for c in stats.calls))
        d.shutdown(); t.join(timeout=0.1)

    # ───────── retry & NS fallback ───────────────────
    def test_retry(self):
        d, t, cfg, _, _ = make_fixture()
        broken = {**cfg}
        broken["services"]["arithmetic"]["direct_uris"]["v1_0"] = ["PYRO:bad@none"]
        with self.assertRaises(main.ServiceError):
            main.ServiceProxy(broken).call("arithmetic", "v1_0", "add", [1, 1])
        d.shutdown(); t.join(timeout=0.1)

    def test_ns_fallback(self):
        d, t, _, px, _ = make_fixture()
        first = {"n": 0}

        def fake_ns(*_, **__):
            first["n"] += 1
            if first["n"] == 1:
                raise Pyro4.errors.PyroError()
            return None  # second attempt → direct URI

        with patch.object(Pyro4, "locateNS", fake_ns):
            self.assertEqual(px.call("arithmetic", "v1_0", "add", [3, 4])["result"], 7)
        d.shutdown(); t.join(timeout=0.1)

    # ───────── heartbeat registration ────────────────
    def test_heartbeat(self):
        d, t, cfg, _, _ = make_fixture()
        holder = main.ConfigHolder(cfg)
        with patch.object(Pyro4.Daemon, "requestLoop", lambda *_: None):
            registry = main.ServiceRegistry(holder)
        captured = []

        class FakeNS:  # noqa: D401
            def register(self, name, uri): captured.append(name)
        registry._ServiceRegistry__ns = lambda: FakeNS()  # type: ignore
        registry._heartbeat()
        self.assertIn("arithmetic.service.v1_0", captured)
        self.assertIn("stats.service.v1_0", captured)

        # graceful shutdown
        registry.d.shutdown()
        d.shutdown(); t.join(timeout=0.1)


if __name__ == "__main__":
    unittest.main(verbosity=2)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/9d506390-ca4f-4c07-a660-bcfe04f0e81f?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67964%2Fview

#### Model code:

```python
import threading
from typing import Any, Union, Optional
import copy

class Config:
    _instance = None
    _lock = threading.RLock()

    def __init__(self, input_json: dict):
        self._input_json = input_json
        self._base = copy.deepcopy(input_json.get("defaults", {}))
        self._json_layer = self._deep_merge(copy.deepcopy(self._base), input_json.get("config_json", {}))
        self._env_layer = self._deep_merge(copy.deepcopy(self._json_layer), self._parse_env_vars(input_json.get("env_vars", {})))
        # <Issue>: CLI overrides are taken as a flat dict, not parsed into nested structure via '__'
        self._overrides = input_json.get("cli_overrides", {})
        # <Issue>: Merging flat CLI overrides directly; nested keys using '__' won’t be expanded
        self._config = self._deep_merge(copy.deepcopy(self._env_layer), self._overrides)

    @classmethod
    def get_instance(cls, input_json: dict) -> 'Config':
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls(input_json)
        return cls._instance

    def _deep_merge(self, dest: dict, src: dict) -> dict:
        for key, value in src.items():
            if isinstance(value, dict) and key in dest and isinstance(dest[key], dict):
                dest[key] = self._deep_merge(dest[key], value)
            else:
                dest[key] = value
        return dest

    def _parse_env_vars(self, env_vars: dict) -> dict:
        result = {}
        for key, value in env_vars.items():
            keys = key.split('__')
            current = result
            for part in keys[:-1]:
                if part not in current or not isinstance(current[part], dict):
                    current[part] = {}
                current = current[part]
            current[keys[-1]] = self._parse_value(value)
        return result

    def _parse_value(self, value: str) -> Union[str, bool]:
        if value.lower() == 'true':
            return True
        elif value.lower() == 'false':
            return False
        return value

    def get(self, key: str, default: Optional[Any] = None) -> Any:
        parts = key.split('.')
        current = self._config
        for part in parts:
            if isinstance(current, dict) and part in current:
                current = current[part]
            else:
                return default
        return current

    def get_int(self, key: str, default: Optional[int] = None) -> Optional[int]:
        value = self.get(key, default)
        if isinstance(value, (int, float)):
            return int(value)
        return default

    def get_bool(self, key: str, default: Optional[bool] = None) -> Optional[bool]:
        value = self.get(key, default)
        if isinstance(value, bool):
            return value
        elif isinstance(value, str):
            return value.lower() == 'true'
        return default

    def reload(self) -> None:
        with self._lock:
            # <Issue>: reload() is incomplete and has a syntax error; does not rebuild layers or swap _config
            self._json_layer = self._deep_merge(copy))

```